# RAG: Process data with llama-index

References
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

## Configuration

In [1]:
class MyConfig:
    pass

MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.INPUT_DATA_DIR = "input_data/walmart-reports-1"

MY_CONFIG.DB_URI = './rag_2_llamaindex.db'
MY_CONFIG.COLLECTION_NAME = 'llamaindex_walmart_docs'

In [2]:
%%time 

from llama_index.core import SimpleDirectoryReader
import pprint

# load documents
documents = SimpleDirectoryReader(
    # input_dir = './data/10k/input/'
    input_dir = MY_CONFIG.INPUT_DATA_DIR
).load_data()

print (f"Loaded {len(documents)} chunks")

# print("Document [0].doc_id:", documents[0].doc_id)
# pprint.pprint (documents[0], indent=4)

Loaded 300 chunks
CPU times: user 7.95 s, sys: 1.5 s, total: 9.45 s
Wall time: 7.42 s


In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/examples/notebooks/rag/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)
    

✅ Connected to Milvus instance:  ./rag_2_llamaindex.db


In [6]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

✅ Connected Llama-index to Milvus instance:  ./rag_2_llamaindex.db


In [7]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
print ("✅ Created index:", index )

✅ Created index: <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x72078bfbaf10>
CPU times: user 4.49 s, sys: 237 ms, total: 4.73 s
Wall time: 4.16 s


In [8]:
%%time 
## Save index to disk
index.storage_context.persist()
print ("✅ Saved documents and data into collection :", MY_CONFIG.COLLECTION_NAME )

✅ Saved documents and data into collection : llamaindex_walmart_docs
CPU times: user 5.43 ms, sys: 2.92 ms, total: 8.35 ms
Wall time: 6.73 ms


In [9]:
# See data in vector db

from pymilvus import MilvusClient
import pprint 

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
res = milvus_client.list_collections()

print(res)
print ('---------')

res = milvus_client.describe_collection(
    collection_name = MY_CONFIG.COLLECTION_NAME
)

pprint.pprint(res)


['llamaindex_walmart_docs']
---------
{'aliases': [],
 'auto_id': False,
 'collection_id': 0,
 'collection_name': 'llamaindex_walmart_docs',
 'consistency_level': 0,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {'max_length': 65535},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             'field_id': 101,
             'name': 'embedding',
             'params': {'dim': 384},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 0,
 'num_shards': 0,
 'properties': {}}
